In [ ]:
import re
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/work/games_seeker/data/raw/juegos.csv", sep="|")

# Completado de datos

In [ ]:
df = df[~(~(df["Sistema operativo"].notnull()) & ~(df["Procesador"].notnull()) & ~(df["Memoria"].notnull()) & ~(df["Gráficos"].notnull()))]

In [ ]:
df["Sistema operativo"][~(df["Sistema operativo"].notnull())] = "Windows 7 de 64-bit"

In [ ]:
df["Sistema operativo"] = df["Sistema operativo"].apply(lambda x: "Windows 7" if "xp" in x.lower() else x)
df["Sistema operativo"] = df["Sistema operativo"].apply(lambda x: "Windows 7" if "vista" in x.lower() else x)

In [ ]:
df["Procesador"][~(df["Procesador"].notnull())] = "i5 / R5"

In [ ]:
df["Memoria"][~(df["Memoria"].notnull())] = "8 GB"

In [ ]:
df["Gráficos"][~(df["Gráficos"].notnull())] = "1060"

In [ ]:
df["Almacenamiento"][~(df["Almacenamiento"].notnull())] = "80 GB"

In [ ]:
df

,name,url,Sistema operativo,Procesador,Memoria,Gráficos,Almacenamiento
0,Elden Ring,https://www.3djuegos.com/34802/elden-ring/,Windows 10 de 64-bit,Intel I5-8400 o AMD Ryzen 5 3 3300X,12 GB de RAM,Nvidia GeForce GTX 1060 3 GB o AMD Radeon RX ...,60 GB de espacio disponible
1,Lost Ark,https://www.3djuegos.com/20774/lost-ark/,Windows 10 de 64-bits,Intel i3 o AMD Ryzen 3,8 GB de RAM,NVIDIA GeForce GTX 460 / AMD HD6850,50 GB de espacio disponible
2,ELEX 2,https://www.3djuegos.com/39382/elex-2/,Windows 10 de 64-bit,AMD Ryzen 3 3100 / Intel Core i5-7400,12 GB de RAM,Radeon RX 5600 XT / Geforce GTX 1060,45 GB de espacio disponible
4,Babylon's Fall,https://www.3djuegos.com/32561/babylons-fall/,Windows 10 de 64-bit,Intel Core i5-7600K / Intel Core i5-10400F / ...,8 GB de RAM,NVIDIA GeForce GTX1650 / NVIDIA GeForce GTX10...,20 GB de espacio disponible
5,Total War: Warhammer 3,https://www.3djuegos.com/38238/total-war-warha...,Windows 7 de 64-bit,Intel i3/Ryzen 3 series,6 GB de RAM,Nvidia GTX 900/AMD RX 400 series | Intel Iris...,120 GB de espacio disponible
...,...,...,...,...,...,...,...
978,Dark Shadows: Army of Evil,https://www.3djuegos.com/14549/dark-shadows-ar...,Windows 7,Intel Core 2 Duo con 2 GHz o similar,2 GB RAM,1060,80 GB
981,404Sight,https://www.3djuegos.com/22131/404sight/,Windows 7/8,Intel Core 2 Quad CPU Q6600,4 GB RAM,1060,80 GB
986,Red Ocean,https://www.3djuegos.com/2341/red-ocean/,Windows 7 de 64-bit,1.8 GHz,512 MB RAM,1060,80 GB
994,Back to Gaya,https://www.3djuegos.com/1572/back-to-gaya/,Windows 7 de 64-bit,1.2 GHz,256 MB RAM,1060,80 GB


# Transformaciones

In [ ]:
df = df.applymap(lambda x: x.strip())

## S.O

In [ ]:
def evaluate_so(x):
    try:
        n = int(re.search(r"\d{1,2}", x).group())
        return n if n < 64 else 7
    except:
        return 7

In [ ]:
df["Sistema operativo"] = df["Sistema operativo"].apply(evaluate_so)

## CPU

In [ ]:
df["Procesador"] = df["Procesador"].apply(lambda x: x.replace("Ryzen ", "R"))

In [ ]:
df["Procesador"] = df["Procesador"].apply(lambda x: x.split(" o ") if " o " in x else x.split("/"))

In [ ]:
def evaluate_cpu(cpus):
    intel = r"i\d{1}"
    ryzen = r"r\d{1}"

    new_cpus = []

    for cpu in cpus:
        cpu = cpu.lower()
        changed = False
        for regex in [intel, ryzen]:
            result = re.search(regex, cpu)
            if result:
                cpu = result.group()
                changed = True
                break
        if changed:
            new_cpus.append(cpu)

    if not new_cpus:
        new_cpus = ["i5", "r5"]      
        
    return new_cpus

In [ ]:
df["Procesador"] = df["Procesador"].apply(evaluate_cpu)

## RAM

In [ ]:
df["Memoria"] = df["Memoria"].apply(lambda x: int(re.search(r"\d{1,2}", x).group()))
df["Memoria"] = df["Memoria"].apply(lambda x: 16 if x > 16 else x)

## Almacenamiento

In [ ]:
df["Almacenamiento"] = df["Almacenamiento"].apply(lambda x: re.search(r"\d{1,3} (GB|MB)", x).group())
df["Almacenamiento"] = df["Almacenamiento"].apply(lambda x: "1 GB" if "MB" in x else x)
df["Almacenamiento"] = df["Almacenamiento"].apply(lambda x: int(x.split()[0]))

## GPU

In [ ]:
df["Gráficos"] = df["Gráficos"].apply(lambda x: x.replace(" o ", "/").replace(",", "/").replace("|", "/").split("/"))

In [ ]:
def evaluate_gpu(gpus):
    regex = r"(GTX*).+(\b\d{3,5})"
    new_gpus = []

    for gpu in gpus:
        gpu = gpu.upper()
        changed = False
        result = re.search(regex, gpu)
        if result:
            gpu = f'{result.group(1)} {result.group(2)}'
            new_gpus.append(gpu)

    if not new_gpus:
        new_gpus = ["GTX 1050"]

    new_gpus = list(map(lambda x: x.split()[-1],new_gpus))
        
    return new_gpus

In [ ]:
df["Gráficos"] = df["Gráficos"].apply(evaluate_gpu)

In [ ]:
df

,name,url,Sistema operativo,Procesador,Memoria,Gráficos,Almacenamiento
0,Elden Ring,https://www.3djuegos.com/34802/elden-ring/,10,"[i5, r5]",12,[1060],60
1,Lost Ark,https://www.3djuegos.com/20774/lost-ark/,10,"[i3, r3]",8,[460],50
2,ELEX 2,https://www.3djuegos.com/39382/elex-2/,10,"[r3, i5]",12,[1060],45
4,Babylon's Fall,https://www.3djuegos.com/32561/babylons-fall/,10,"[i5, i5, r3, r5]",8,[1050],20
5,Total War: Warhammer 3,https://www.3djuegos.com/38238/total-war-warha...,7,"[i3, r3]",6,[900],120
...,...,...,...,...,...,...,...
978,Dark Shadows: Army of Evil,https://www.3djuegos.com/14549/dark-shadows-ar...,7,"[i5, r5]",2,[1050],80
981,404Sight,https://www.3djuegos.com/22131/404sight/,7,"[i5, r5]",4,[1050],80
986,Red Ocean,https://www.3djuegos.com/2341/red-ocean/,7,"[i5, r5]",16,[1050],80
994,Back to Gaya,https://www.3djuegos.com/1572/back-to-gaya/,7,"[i5, r5]",16,[1050],80


In [ ]:

df.to_csv("/work/games_seeker/data/interim/games_requirements.csv", sep="|", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=07e29549-6cbf-4e88-b69c-0fc48f2d6023' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>